# Data Collection


Here you are asked to collect data to then guess the model of the Duckiebot


In [ ]:
from lraClass import LRA2_HELPER

import time
import sys
import argparse
import math
import numpy as np
import gym
from simulator.src.gym_duckietown.simulator import Simulator


Enviroment initialization

In [ ]:
lra = LRA2_HELPER()


env = Simulator(
    map_name = "udem1",
    domain_rand = False,
    draw_bbox = False
)

env = lra.wrapEnv(env)

env.reset()


Simulate the environment. Here you can collect the state and the control inputs.

In the Duckietown environment the control input **_u_** is the steering angle, while the state **_X_** is represented by the distance from the center of the lane and the angle from the straight as shown in the image below

![controls](assets/control_model.png)



In [ ]:

#
#   Simulate the environment with a PID controller 
#
speed = 0.2
while True:
    lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
    distance_to_road_center = lane_pose.dist
    angle_from_straight_in_rads = lane_pose.angle_rad

    k_p = 10
    k_d = 1

    X = [distance_to_road_center, angle_from_straight_in_rads]
    steering =  k_p * distance_to_road_center + k_d * angle_from_straight_in_rads

    ### YOUR CODE HERE
    ### Collect the data here using the lra class

    obs, reward, done, info = env.step([speed, steering])

    #print('Steps = %s' % (env.step_count))

    

    if done:
        if reward < 0:
            print('*** CRASHED ***')
        break


### Don't forget to save the data!

env.close()

lra.renderEnv(env)

In the cell below run a random controll inputs and collect the data as in the previous exercise 

In [ ]:

#
#   Simulate the environment with a random controller 
#
speed = 0.2
while True:
    lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
    distance_to_road_center = lane_pose.dist
    angle_from_straight_in_rads = lane_pose.angle_rad

    X = [distance_to_road_center, angle_from_straight_in_rads]

    ### YOUR CODE HERE
    ### Collect the data here using the lra class
    steering = 


    obs, reward, done, info = env.step([speed, steering])

    #print('Steps = %s' % (env.step_count))

    

    if done:
        if reward < 0:
            print('*** CRASHED ***')
        break


### Don't forget to save the data!

env.close()

lra.renderEnv(env)

# Model estimation and Data cleaning


In [ ]:
from sklearn.linear_model import Ridge
import pandas as pd

You are asked to estimate the dynamics model of a Duckiebot. We can approximate the dynamics of a Duckiebot with a linear system:

$$
x(t+1) = A \cdot x(t) + B \cdot u(t)
$$


The goal here is to guess the values of the matrices A and B.

_Hint: linear regression problem_


Divide your dataset in test and training:

In [ ]:
### Clean the data before splitting the dataset

data = lra.loadData("model_data.csv")



Train the linear model on the train set:

In [ ]:


##  YOR CODE HERE



Check how good is the model you got according to your test set.

# LQR design

Change the **_A_** and **_B_** matrices with the model you found the previous section. Then try to tune the **_Q_** and **_R_** matrices in order to have good driving performance.

The function LQRGain returns the **_K_** gain needed to realize the state feedback controller.

$\color{red}{\text{DO NOT CHANGE THE NAME OF THIS FUNCTION.}}$


In [ ]:

#
#   Find the state feedback control gain
#
def LQRGain():
    from lraClass import LRA2_HELPER
    import numpy as np
    
    lra = LRA2_HELPER()

    ### Write the A and B matrices you found in the previous section.
    A = np.array([[0.0, 0.0 ],
            [0.0, 0.0 ]])

    B = np.array([[0.0, 0.0]]).T

    Q = np.array([[0.0, 0.0], [0.0, 0.0]])
    R = np.array([[0.0]])


    # Solve Algebric Riccati Equation
    S = lra.solveRiccati(A,B,Q,R)

    # find the gain K
    K = 

    return K


In [ ]:

speed=0.2


K=LQRGain()

env = lra.wrapEnv(env)
env.reset()

#
#   Simulate the environment
#
while True:
    lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
    distance_to_road_center = lane_pose.dist
    angle_from_straight_in_rads = lane_pose.angle_rad

    X = [distance_to_road_center, angle_from_straight_in_rads]

    # YOUR CODE HERE
    # find the steering using the gain you found.
    steering = 
    
    obs, reward, done, info = env.step([speed, steering])

    #print('Steps = %s' % (env.step_count))

    if done:
        if reward < 0:
            print('*** CRASHED ***')
        break

env.close()

lra.renderEnv(env)